## Conversion to fif: BJH024

This script takes the raw, original data file we recieved from WashU and converts it to a fif file with the photodiode as a new stim channel


Just need to change fields in `prep_paths`

In [7]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
from BCI2kReader import BCI2kReader as b2k
import os
import mne
from tabulate import tabulate
import IPython
import seaborn as sns

In [8]:
## prep paths

subject = 'BJH024'
orig_data_fi = '/home/brooke/knight_server/remote/WashU/data/PacmanTask/BJH024/PacmanTask/ECOG001/ECOGS001R01.dat'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}/ieeg"

In [9]:
## load data

with b2k.BCI2kReader(orig_data_fi) as data: #opens a stream to the dat file
    my_signals = data.signals
    my_states = data.states
    my_params = data.parameters

In [10]:
my_params

{'SamplingRate': 30000,
 'SourceCh': 21,
 'SampleBlockSize': 3000,
 'ChannelNames': ['chan1',
  'chan2',
  'chan3',
  'chan4',
  'chan5',
  'chan6',
  'chan7',
  'chan8',
  'chan9',
  'chan10',
  'chan11',
  'chan12',
  'chan13',
  'chan14',
  'chan15',
  'chan16',
  'ainp1',
  'ainp2',
  'ainp3',
  'ainp4',
  'sync'],
 'SourceChOffset': ['0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 'SourceChGain': ['0.25',
  '0.25',
  '0.25',
  '0.25',
  '0.25',
  '0.25',
  '0.25',
  '0.25',
  '0.25',
  '0.25',
  '0.25',
  '0.25',
  '0.25',
  '0.25',
  '0.25',
  '0.25',
  '0.152593',
  '0.152593',
  '0.152593',
  '0.152593',
  '0.152593'],
 'DataDirectory': '../data/BJH/PacmanTask',
 'SubjectName': 'ECOG',
 'SubjectSession': '001',
 'SubjectRun': '01',
 'ID_System': '',
 'ID_Amp': '',
 'ID_Montage': '',
 'VisualizeTiming': 1,
 'VisualizeSource': 1,
 'VisualizeSourceDecimation': None,
 'VisualizeSource

In [ ]:
## check structure if necessary

# my_states
# my_params

In [5]:
## create info field 

info = mne.create_info(ch_names = my_params['ChannelNames'],
                       sfreq = my_params['SamplingRate'],
                       ch_types = 'seeg')

info

<Info | 7 non-empty values
 bads: []
 ch_names: AL1, AL2, AL3, AL5, REF1, REF2, AL6, AL7, AL8, AL9, AL11, AL12, ...
 chs: 256 sEEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 1000.0 Hz
 meas_date: unspecified
 nchan: 256
 projs: []
 sfreq: 2000.0 Hz
>

In [6]:
## create mne object
mne_df = mne.io.RawArray(my_signals, info)

Creating RawArray with float64 data, n_channels=256, n_times=4365200
    Range : 0 ... 4365199 =      0.000 ...  2182.599 secs
Ready.


In [7]:
## create stim channel and add it to mne object

info = mne.create_info(['STI'], my_params['SamplingRate'], ['stim'])
stim_raw = mne.io.RawArray( my_states['Trial_on_off'], info)
mne_df.add_channels([stim_raw], force_update_info=True)

Creating RawArray with float64 data, n_channels=1, n_times=4365200
    Range : 0 ... 4365199 =      0.000 ...  2182.599 secs
Ready.


<RawArray | 257 x 4365200 (2182.6 s), ~8.36 GB, data loaded>

In [8]:
## save it

mne_df.save(f"{raw_data_dir}/{subject}_raw_ieeg.fif", overwrite = False)

Writing /home/brooke/pacman/raw_data/BJH024/ieeg/BJH024_raw_ieeg.fif
Overwriting existing file.
Writing /home/brooke/pacman/raw_data/BJH024/ieeg/BJH024_raw_ieeg-1.fif
Overwriting existing file.
Writing /home/brooke/pacman/raw_data/BJH024/ieeg/BJH024_raw_ieeg-2.fif
Closing /home/brooke/pacman/raw_data/BJH024/ieeg/BJH024_raw_ieeg-2.fif
Closing /home/brooke/pacman/raw_data/BJH024/ieeg/BJH024_raw_ieeg-1.fif
Closing /home/brooke/pacman/raw_data/BJH024/ieeg/BJH024_raw_ieeg.fif
[done]
